# 13.1 Interfacing Between pandas and Model Code
The point of contact between pandas and other analysis libraries is usually NumPy arrays.<mark> To turn a DataFrame into a NumPy array, use the `.values` property.</mark>

In [1]:
import pandas as pd
import numpy as np

data = pd.DataFrame({
    'x0': [1,2,3,4,5],
    'x1': [0.01, 0.02, 0.03, 0.04, 0.05],
    'x2': [-1.5, 0., 3.6, 1.3, -2.]
})

data

,x0,x1,x2
0,1,0.01,-1.5
1,2,0.02,0.0
2,3,0.03,3.6
3,4,0.04,1.3
4,5,0.05,-2.0


In [2]:
data.columns

Index(['x0', 'x1', 'x2'], dtype='object')

In [3]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  ,  0.02,  0.  ],
       [ 3.  ,  0.03,  3.6 ],
       [ 4.  ,  0.04,  1.3 ],
       [ 5.  ,  0.05, -2.  ]])

To convert back to DataFrame

In [4]:
df2 = pd.DataFrame(data.values, columns=['one', 'two', 'three'])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,0.02,0.0
2,3.0,0.03,3.6
3,4.0,0.04,1.3
4,5.0,0.05,-2.0


The `.values` attribute is intended to be used when you data is homogeneous-for example, all numeric type. If you have heterogeneous data, the result will be an ndarray of Python objects.

In [5]:
df3 = data.copy()
df3['strings'] = ['a', 'b', 'c', 'd', 'e']
df3

,x0,x1,x2,strings
0,1,0.01,-1.5,a
1,2,0.02,0.0,b
2,3,0.03,3.6,c
3,4,0.04,1.3,d
4,5,0.05,-2.0,e


In [7]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, 0.02, 0.0, 'b'],
       [3, 0.03, 3.6, 'c'],
       [4, 0.04, 1.3, 'd'],
       [5, 0.05, -2.0, 'e']], dtype=object)

To use subset of columns/rows; use `.loc`.

In [8]:
model_cold = ['x0', 'x1']
data.loc[:, model_cold].values

array([[1.  , 0.01],
       [2.  , 0.02],
       [3.  , 0.03],
       [4.  , 0.04],
       [5.  , 0.05]])

Suppose we had a nonumeric column in our dataset.

In [9]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'], categories=['a', 'b'])
data

,x0,x1,x2,category
0,1,0.01,-1.5,a
1,2,0.02,0.0,b
2,3,0.03,3.6,a
3,4,0.04,1.3,a
4,5,0.05,-2.0,b


In [10]:
# We might need to add dummy category varaible
dummies = pd.get_dummies(data.category, prefix='category')
data_with_dummies = data.drop('category', axis=1).join(dummies)

data_with_dummies

,x0,x1,x2,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,0.02,0.0,0,1
2,3,0.03,3.6,1,0
3,4,0.04,1.3,1,0
4,5,0.05,-2.0,0,1


# 13.2 Creating Model Description with Patsy
Patsy is a Python library for describing statistical models (especially linear models) with a small string-based "formula syntax"
### `patsy.dmatrices`
Tkaes a formula string along with a dataset (which can be a DataFrame or a dict) and produces design matrices for a linear model.

In [3]:
data = pd.DataFrame({
    'x0': [1, 2, 3, 4, 5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.],
    'y': [-1.5, 0., 3.6, 1.3, -2.]
})

data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [4]:
import patsy

y, X = patsy.dmatrices('y ~ x0 + x1', data)
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [28]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

These Patsy `DesignMatrix` instances are NumPy ndarrays with additional metadata:

In [29]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [30]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

`Intercept` term is a convension for linear models like Ordinary least squares(OLS) regression. You can suppress this by adding a `0` to the model.

In [31]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

Patsy objects can be passed directly into algorithm like `numpy.linalg.lstsq`, which performs an ordinary least squares regression.

In [33]:
coef, resid, _, _ = np.linalg.lstsq(X, y, rcond=None)
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

The <mark>model metadata is retained in</mark> `design_info` attribute, so you can reattach the model column names to the fitted coefficients to obatain a Series.

In [35]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

## Data Transformation in Pasty Formulas
You can mix Python code into your Patsy Formulas;

In [36]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

Patsy has built-in functions for some common used varaible transformations like **standardizing** <mark>(to mean 0 and variance 1)</mark> and **centering** <mark>(substracting the mean)</mark>.

In [5]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

The `patsy.build_design_matrices` function can <mark>apply transformations to new *out-of-sample* data usng the saved information from the original *in-sample* dataset.</mark>

In [6]:
new_data = pd.DataFrame({
    'x0': [6, 7, 8, 9],
    'x1': [4.1, -0.5, 0., 2.3],
    'y': [1, 2, 3, 4]
})

new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        4.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

Because the plus symbol in the context of Patsy Formula does not mean addition when you want to add columns from a dataset by name, <mark>you must wrap them in the special $I$ function.</mark>

In [7]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

Patsy has several other built-in transforms in the `patsy.builtins` module. See the online documentation.

## Categorical Data and Patsy
When you use non-numeric terms in a Patsy formula,<mark> they are converted to dummy varaibles by default.</mark> If there is an intercept, one of the levels will be left out to avoid collinearity.

In [8]:
data = pd.DataFrame({
    'key1': ['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2': [0, 1, 0, 1, 0, 1, 0, 0],
    'v2': [-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})

y, X = patsy.dmatrices('v2 ~ key1', data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

<mark>If you omit the intercept</mark> from the model, <mark>then columns for each category value will be included</mark> in the model design matrix:

In [11]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

<mark>Numeric columns</mark> can be <mark>interpreted as categorical</mark> with `C` function:

In [14]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

You can <mark>include interaction terms of the form`key1:key2`</mark>, which can be used in say, <mark>analysis of variance (ANOVA)</mark> models:

In [15]:
data['key2'] = data['key2'].map({0: 'zero', 1: 'one'})
data

,key1,key2,v2
0,a,zero,-1.0
1,a,one,0.0
2,b,zero,2.5
3,b,one,-0.5
4,a,zero,4.0
5,b,one,-1.2
6,a,zero,0.2
7,b,zero,-1.7


In [16]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [17]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

# 13.2 Introduction to statsmodels
statsmodels is a Python library for fitting many kinds of statistical models, performing statistical test, and data exploration and visualisation.

Next up, we will use a few basic tools in statsmodels and explore how to use the modeling interfaces with Patsy formulas and pandas DataFrame objects.

### Estimating Linear Models
There are several kinds of linear models in statsmodels, and have <mark>two different main interface</mark>:
1. **Array-based**
2. **Formula-based**

In [19]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

To show how to use these, we generate a linear model from some random data:

**Note:**
1. Inside a **function header**
    - `*` collects all the poisitional arguments in a tuple.
    
    - `**` collects all the keyword arguments in a dictionary.
    - `>>> def functionA(*a, **kw):
        print(a)
        print(kw)`
        
        `>>> functionA(1, 2, 3, a=2, b=3, c=5)
        (1, 2, 3)
        {'a': 2, 'b': 3, 'c': 5}`
2. Inside a **function call**
   - `*` unpacks a list or tuple into position arguments.
   - `**` unpacks a dictionary into keyword arguments.
   - `>>> lis = [1, 2, 3]`
   
       `>>> dic = {'a': 10, 'b': 20}`
   
       `>>> functionA(*lis, **dic)
       (1, 2, 3)
       {'a': 10, 'b': 20}`
        

In [32]:
# "true" model with known parameters beta

def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size=size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

# For reproducibility
np.random.seed(1234)

N = 100
X = np.c_[dnorm(0, 0.4, size=N),
          dnorm(0, 0.6, size=N),
          dnorm(0, 0.2, size=N)]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

In this case,`dnorm` is a helper function for <mark>generating normally distributed data with particular mean and variance.</mark>

In [33]:
X[:5]

array([[ 0.29816178,  0.2255667 , -0.1429122 ],
       [-0.75323917,  0.43883511, -0.27726934],
       [ 0.90612345,  0.3900805 ,  0.07021181],
       [-0.19773842,  0.22098909, -0.2555626 ],
       [-0.45574033,  0.37512796,  0.47298794]])

A linear model is generally fitted with an intercept term as we saw before with Patsy. 
### `sm.add_constant` 
Function can <mark>add an intercept column to an existing matrix.</mark>

In [34]:
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.        ,  0.29816178,  0.2255667 , -0.1429122 ],
       [ 1.        , -0.75323917,  0.43883511, -0.27726934],
       [ 1.        ,  0.90612345,  0.3900805 ,  0.07021181],
       [ 1.        , -0.19773842,  0.22098909, -0.2555626 ],
       [ 1.        , -0.45574033,  0.37512796,  0.47298794]])

The `sm.OLS` class can <mark>fit an ordinary least squares linear regression</mark>

In [35]:
model = sm.OLS(y, X)

The model's `fit` method returns a regression results object containing estimated model parameters and other diagnostics:

In [36]:
results = model.fit()
results.params

array([0.09185321, 0.25312566, 0.40497531])

The `summary` method on `results` can <mark>print a model detailing diagnostic output of the model</mark>:

In [37]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.468
Model:                            OLS   Adj. R-squared (uncentered):              0.452
Method:                 Least Squares   F-statistic:                              28.49
Date:                Tue, 02 Mar 2021   Prob (F-statistic):                    2.68e-13
Time:                        15:56:47   Log-Likelihood:                         -20.588
No. Observations:                 100   AIC:                                      47.18
Df Residuals:                      97   BIC:                                      54.99
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

The parameter here have been given the generic names. What if instead that all of the model parameter are in a DataFrame?

In [38]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])
data['y'] = y
data[:5]

,col0,col1,col2,y
0,0.298162,0.225567,-0.142912,0.192697
1,-0.753239,0.438835,-0.277269,0.142565
2,0.906123,0.390080,0.070212,0.173948
3,-0.197738,0.220989,-0.255563,0.752884
4,-0.455740,0.375128,0.472988,-0.247454


Now we can use the statsmodels formula API and Patsy formula strings:

In [39]:
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()
results.params

Intercept    0.039247
col0         0.088451
col1         0.257012
col2         0.389228
dtype: float64

In [40]:
results.tvalues

Intercept    1.282929
col0         1.836565
col1         6.396628
col2         5.378643
dtype: float64

Observe how statsmodels has returned results as Series with DataFrame names attached. We calso don't need to use `add_constant` when using formulas and pandas objects.

Given new out-of-sample data, you can compute predicted values given the estimated model parameters.

In [41]:
results.predict(data[:5])

0    0.067967
1   -0.022513
2    0.246978
3   -0.020919
4    0.279448
dtype: float64

## Estimating Time Series Processes
let's simulate some time series data with an autoregressive structure and noise:

In [43]:
import random

init_x = 4
values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

This data has an <mark>AR(2) structure (two *lags*)</mark> with <mark>parameter 0.8 and 0.4</mark>. When you fit an AR model, you may not know the number of lagged terms to include, so you can fit the model with some larger number of flags.

In [44]:
MAXFLAGS = 5
model = sm.tsa.AR(values)
results = model.fit(MAXFLAGS)

C:\anaconda3\lib\site-packages\statsmodels\tsa\ar_model.py:791: FutureWarning: 
statsmodels.tsa.AR has been deprecated in favor of statsmodels.tsa.AutoReg and
statsmodels.tsa.SARIMAX.

AutoReg adds the ability to specify exogenous variables, include time trends,
and add seasonal dummies. The AutoReg API differs from AR since the model is
treated as immutable, and so the entire specification including the lag
length must be specified when creating the model. This change is too
substantial to incorporate into the existing AR api. The function
ar_select_order performs lag length selection for AutoReg models.

AutoReg only estimates parameters using conditional MLE (OLS). Use SARIMAX to
estimate ARX and related models using full MLE via the Kalman Filter.

To silence this warning and continue using AR until it is removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.ar_model.AR', FutureWarning)

  warnings.warn(AR_DEPRECATION_WARN, FutureWarning)


In [45]:
results.params

array([ 0.00082601,  0.75515536, -0.42265735,  0.04407968,  0.0051507 ,
        0.01544671])